In [2]:
import cv2
import mediapipe as mp
import math
import serial
import pygame
import os
import time

# Initialize music playback
pygame.mixer.init()
# Path to the music folder
music_folder = "music"
music_files = [f for f in os.listdir(music_folder) if f.endswith(".mp3")]
current_index = 0  # Current music index being played

def sendSongName(song_name):
    arduino.write(f"{song_name}\n".encode())

def play_music(index):
    global current_index
    current_index = index % len(music_files)  # Prevent the index from going out of range
    file_path = os.path.join(music_folder, music_files[current_index])
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()
    print(f"Playing: {music_files[current_index]}")

    sendSongName(music_files[current_index])

def nextSong():
    play_music(current_index + 1)

def previousSong():
    play_music(current_index - 1)

# Initialize serial communication (for communication with Arduino)
arduino = serial.Serial(port='your_port', baudrate=9600, timeout=1)  # Replace with the actual COM port
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

def calculateDistance(point1, point2):
    return math.sqrt((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2)

def getAction(action):
    """
    Perform actions based on detected gestures:
    - Send corresponding commands to Arduino
    - Display the current status
    """
    if action == "Pause":
        print("Sending: Pause")
        arduino.write(b"Pause\n")
        pygame.mixer.music.pause()
        return "Pause"
        
    elif action == "Play":
        print("Sending: Play")
        arduino.write(b"Play\n")
        pygame.mixer.music.unpause()
        return "Play"
 
    elif action == "Next":
        print("Sending: Next")
        arduino.write(b"Next\n")
        nextSong()
        return "Next"
 
    elif action == "Previous":
        print("Sending: Previous")
        arduino.write(b"Previous\n")
        previousSong()
        return "Previous"

def setVolume(volume_value):
    """
    Set the volume and convert the value to a range between 0.0 and 1.0
    volume_value range: 0 - 100
    """
    volume = volume_value / 100.0  # Convert volume value to a range of 0.0 to 1.0
    pygame.mixer.music.set_volume(volume)

def readVolumeFromArduino():
    """
    Read volume data from Arduino and adjust the volume
    """
    if arduino.in_waiting > 0:  # Check if there is data available to read
        line = arduino.readline().decode('utf-8').strip()
        if line.startswith("Volume:"):
            try:
                # Parse volume value in the range 0 - 100
                volume_value = int(line.split(":")[1].strip())
                setVolume(volume_value)  # Adjust volume
            except ValueError:
                print("Invalid volume value received.")

# Set cooldown time (in seconds)
cooldown_time = 2  # 1 second cooldown time
last_gesture_time = time.time()  # Time of the last gesture detection

# Open the camera
cap = cv2.VideoCapture(0)
current_action = ""  # Current action
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Get landmarks
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            # Calculate the distance between thumb and index finger
            thumb_index_distance = calculateDistance(thumb_tip, index_finger_tip)
            thumb_middle_distance = calculateDistance(thumb_tip, middle_finger_tip)

            # Detect OK gesture (distance between thumb and index finger is within a certain range)
            ok_gesture_detected = thumb_index_distance < 0.1  # Adjust as needed

            # Detect open palm and check distances for all fingers
            open_hand_detected = all(calculateDistance(hand_landmarks.landmark[0], hand_landmarks.landmark[i]) > 0.2 for i in [4, 8, 12, 16, 20])

            # Detect gestures and perform corresponding actions
            current_time = time.time()
            if current_time - last_gesture_time >= cooldown_time:  # Check if cooldown time has passed
                if ok_gesture_detected:
                    current_action = getAction("Pause")  # OK gesture: pause music
                    last_gesture_time = current_time  # Update last detection time
                
                elif open_hand_detected:
                    current_action = getAction("Play")  # Open palm: play music
                    last_gesture_time = current_time  # Update last detection time

                elif calculateDistance(index_finger_tip, thumb_tip) > 0.1 and calculateDistance(middle_finger_tip, thumb_tip) > 0.1:
                    current_action = getAction("Next")  # Two fingers: next song
                    last_gesture_time = current_time  # Update last detection time

                elif calculateDistance(index_finger_tip, thumb_tip) > 0.1:
                    current_action = getAction("Previous")  # One finger: previous song
                    last_gesture_time = current_time  # Update last detection time

    # Display the current action and song name
    font = cv2.FONT_HERSHEY_SIMPLEX
    action_text = f"Action: {current_action}"
    song_text = f"Song: {music_files[current_index]}"

    # Show action and song name on the screen
    cv2.putText(frame, action_text, (8, 30), font, 0.85, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, song_text, (8, 70), font, 0.85, (255, 255, 0), 2, cv2.LINE_AA)

    # Check for volume data from Arduino in each loop iteration
    readVolumeFromArduino()

    cv2.imshow("Hand Gesture Control", frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
arduino.close()

Sending: Previous
Playing: VERNON Black Eye.mp3
